<a href="https://colab.research.google.com/github/TheCaveOfAdullam/study3/blob/main/gradientTest7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Drive 연결 및 필요한 라이브러리 불러오기
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install tensorflow_model_optimization

In [5]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder
import tensorflow_model_optimization as tfmot

In [7]:
# 기본 경로 설정
base_dir = '/content/drive/MyDrive/ship_motor10'
categories = ['normal', 'fault_BB', 'fault_RI', 'fault_SM']

# 데이터 로드 및 전처리 함수 정의
def load_data(base_dir, split):
    X = []
    y = []
    split_dir = os.path.join(base_dir, split)
    for category in categories:
        category_dir = os.path.join(split_dir, category)
        for file in os.listdir(category_dir):
            file_path = os.path.join(category_dir, file)
            data = pd.read_csv(file_path, header=None).values
            data = pd.to_numeric(data.flatten(), errors='coerce').reshape(-1, data.shape[1])
            data = np.nan_to_num(data).astype('float32')  # NaN 값을 0으로 대체하고, float32로 변환
            X.append(data)
            y.append(category)
    return np.array(X), np.array(y)

# 데이터 로드
X_train, y_train = load_data(base_dir, 'train')
X_val, y_val = load_data(base_dir, 'validation')
X_test, y_test = load_data(base_dir, 'test')

# 데이터 차원 변경 (CNN 입력 형식에 맞게)
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# 레이블 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# 원-핫 인코딩
y_train_categorical = tf.keras.utils.to_categorical(y_train_encoded)
y_val_categorical = tf.keras.utils.to_categorical(y_val_encoded)
y_test_categorical = tf.keras.utils.to_categorical(y_test_encoded)

In [8]:
# 모델 정의
model = models.Sequential()
model.add(layers.Conv1D(filters=64, kernel_size=16, strides=16, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(layers.MaxPooling1D(pool_size=2, strides=2))
model.add(layers.Conv1D(filters=32, kernel_size=3, strides=1, activation='relu'))
model.add(layers.Conv1D(filters=64, kernel_size=5, strides=1, activation='relu'))
model.add(layers.Conv1D(filters=128, kernel_size=5, strides=1, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2, strides=2))
model.add(layers.Flatten())
model.add(layers.Dense(units=5000, activation='relu'))
model.add(layers.Dense(units=1000, activation='relu'))
model.add(layers.Dense(len(categories), activation='softmax'))

# 모델 컴파일
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 750, 64)           2112      
                                                                 
 max_pooling1d (MaxPooling1  (None, 375, 64)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 373, 32)           6176      
                                                                 
 conv1d_2 (Conv1D)           (None, 369, 64)           10304     
                                                                 
 conv1d_3 (Conv1D)           (None, 365, 128)          41088     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 182, 128)          0         
 g1D)                                                   

In [9]:
# 모델 훈련
history = model.fit(X_train, y_train_categorical, epochs=10, validation_data=(X_val, y_val_categorical))

Epoch 1/10
394/394 [==============================] - 229s 575ms/step - loss: 0.9432 - accuracy: 0.6245 - val_loss: 0.3934 - val_accuracy: 0.7567
Epoch 2/10
394/394 [==============================] - 223s 567ms/step - loss: 0.3288 - accuracy: 0.8508 - val_loss: 0.1119 - val_accuracy: 0.9652
Epoch 3/10
394/394 [==============================] - 227s 576ms/step - loss: 0.2633 - accuracy: 0.8994 - val_loss: 0.4285 - val_accuracy: 0.7922
Epoch 4/10
394/394 [==============================] - 229s 582ms/step - loss: 0.0603 - accuracy: 0.9782 - val_loss: 0.0060 - val_accuracy: 0.9989
Epoch 5/10
394/394 [==============================] - 226s 574ms/step - loss: 0.0186 - accuracy: 0.9927 - val_loss: 0.0277 - val_accuracy: 0.9904
Epoch 6/10
394/394 [==============================] - 224s 569ms/step - loss: 0.0886 - accuracy: 0.9789 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 7/10
394/394 [==============================] - 224s 567ms/step - loss: 0.0208 - accuracy: 0.9931 - val_loss: 2.9990e-

In [10]:
# 모델 평가
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical)
print(f'Test accuracy before pruning: {test_accuracy}')

85/85 [==============================] - 3s 40ms/step - loss: 0.0069 - accuracy: 0.9981
Test accuracy before pruning: 0.9981481432914734


In [11]:
# 원본 모델 저장
model.save('model_before_pruning.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
# --- 매그니튜드 기반 프루닝 ---
def magnitude_based_pruning(model, pruning_threshold=0.01):
    for layer in model.layers:
        if hasattr(layer, 'kernel'):
            weights, biases = layer.get_weights()
            weights[np.abs(weights) < pruning_threshold] = 0
            biases[np.abs(biases) < pruning_threshold] = 0
            layer.set_weights([weights, biases])

In [13]:
# --- 변화도 기반 프루닝 ---
def gradient_based_unstructured_pruning(model, X, y, pruning_threshold=0.0009):
    loss_fn = tf.keras.losses.CategoricalCrossentropy()
    with tf.GradientTape() as tape:
        predictions = model(X, training=True)
        loss = loss_fn(y, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    grad_index = 0
    for layer in model.layers:
        if hasattr(layer, 'kernel'):
            weights, biases = layer.get_weights()
            weight_grad = grads[grad_index]
            bias_grad = grads[grad_index + 1]
            grad_index += 2
            weight_grad_abs = np.abs(weight_grad)
            bias_grad_abs = np.abs(bias_grad)
            weights[weight_grad_abs < pruning_threshold] = 0
            biases[bias_grad_abs < pruning_threshold] = 0
            layer.set_weights([weights, biases])

In [14]:
# --- 비제로 가중치 계산 함수 ---
def calculate_non_zero_weights(model):
    total_weights = 0
    non_zero_weights = 0
    for layer in model.layers:
        if hasattr(layer, 'kernel'):
            weights, biases = layer.get_weights()
            total_weights += np.prod(weights.shape) + np.prod(biases.shape)
            non_zero_weights += np.count_nonzero(weights) + np.count_nonzero(biases)
    return non_zero_weights, total_weights

In [15]:
# --- 1차 매그니튜드 기반 프루닝 ---
magnitude_based_pruning(model, pruning_threshold=0.01)
history_pruned_1 = model.fit(X_train, y_train_categorical, epochs=10, validation_data=(X_val, y_val_categorical))

Epoch 1/10
394/394 [==============================] - 229s 582ms/step - loss: 0.3032 - accuracy: 0.9857 - val_loss: 2.3641e-04 - val_accuracy: 1.0000
Epoch 2/10
394/394 [==============================] - 228s 578ms/step - loss: 0.0026 - accuracy: 0.9990 - val_loss: 1.1254e-04 - val_accuracy: 1.0000
Epoch 3/10
394/394 [==============================] - 229s 582ms/step - loss: 0.0603 - accuracy: 0.9830 - val_loss: 1.1099e-05 - val_accuracy: 1.0000
Epoch 4/10
394/394 [==============================] - 232s 588ms/step - loss: 0.0045 - accuracy: 0.9987 - val_loss: 0.0019 - val_accuracy: 0.9996
Epoch 5/10
394/394 [==============================] - 234s 593ms/step - loss: 0.0434 - accuracy: 0.9833 - val_loss: 3.8102e-04 - val_accuracy: 1.0000
Epoch 6/10
394/394 [==============================] - 233s 591ms/step - loss: 0.0173 - accuracy: 0.9938 - val_loss: 8.5166e-05 - val_accuracy: 1.0000
Epoch 7/10
394/394 [==============================] - 234s 594ms/step - loss: 0.1708 - accuracy: 0.9683 

In [16]:
# --- 2차 변화도 기반 프루닝 ---
gradient_based_unstructured_pruning(model, X_train[:100], y_train_categorical[:100])
history_pruned_2 = model.fit(X_train, y_train_categorical, epochs=10, validation_data=(X_val, y_val_categorical))

Epoch 1/10
394/394 [==============================] - 238s 604ms/step - loss: 1.2592 - accuracy: 0.4995 - val_loss: 1.2443 - val_accuracy: 0.5000
Epoch 2/10
394/394 [==============================] - 235s 596ms/step - loss: 1.2448 - accuracy: 0.5000 - val_loss: 1.2430 - val_accuracy: 0.5000
Epoch 3/10
394/394 [==============================] - 237s 601ms/step - loss: 1.2441 - accuracy: 0.5000 - val_loss: 1.2430 - val_accuracy: 0.5000
Epoch 4/10
394/394 [==============================] - 236s 598ms/step - loss: 1.2439 - accuracy: 0.5000 - val_loss: 1.2427 - val_accuracy: 0.5000
Epoch 5/10
394/394 [==============================] - 235s 598ms/step - loss: 1.2439 - accuracy: 0.5000 - val_loss: 1.2450 - val_accuracy: 0.5000
Epoch 6/10
394/394 [==============================] - 235s 596ms/step - loss: 1.2425 - accuracy: 0.5000 - val_loss: 1.2409 - val_accuracy: 0.5000
Epoch 7/10
394/394 [==============================] - 234s 594ms/step - loss: 1.1552 - accuracy: 0.5188 - val_loss: 0.6059 -

In [17]:
# --- 구조적 프루닝: TF-MOT 사용 ---
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0,
        final_sparsity=0.5,
        begin_step=2000,
        end_step=10000
    )
}

In [18]:
# 프루닝 가능한 모델로 변환
model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)
model_for_pruning.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
# 프루닝 콜백 설정
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir='/tmp/pruning_logs')
]

In [20]:
# 모델 재훈련 (구조적 프루닝 적용)
history_pruned_structured = model_for_pruning.fit(
    X_train, y_train_categorical,
    epochs=10,
    validation_data=(X_val, y_val_categorical),
    callbacks=callbacks
)

Epoch 1/10
394/394 [==============================] - 239s 596ms/step - loss: 0.3777 - accuracy: 0.9045 - val_loss: 0.5088 - val_accuracy: 0.7174
Epoch 2/10
394/394 [==============================] - 241s 611ms/step - loss: 0.1013 - accuracy: 0.9719 - val_loss: 0.0058 - val_accuracy: 0.9993
Epoch 3/10
394/394 [==============================] - 240s 609ms/step - loss: 0.1384 - accuracy: 0.9602 - val_loss: 0.0136 - val_accuracy: 0.9996
Epoch 4/10
394/394 [==============================] - 240s 610ms/step - loss: 8.3903e-04 - accuracy: 0.9999 - val_loss: 2.2141e-04 - val_accuracy: 1.0000
Epoch 5/10
394/394 [==============================] - 240s 610ms/step - loss: 1.2498e-04 - accuracy: 1.0000 - val_loss: 5.1729e-05 - val_accuracy: 1.0000
Epoch 6/10
394/394 [==============================] - 297s 755ms/step - loss: 5.2195e-05 - accuracy: 1.0000 - val_loss: 2.7153e-05 - val_accuracy: 1.0000
Epoch 7/10
394/394 [==============================] - 300s 763ms/step - loss: 2.5759e-05 - accuracy:

In [21]:
# 모델 평가
test_loss_pruned, test_accuracy_pruned = model_for_pruning.evaluate(X_test, y_test_categorical)
print(f'Test accuracy after structured pruning: {test_accuracy_pruned}')

85/85 [==============================] - 11s 125ms/step - loss: 2.2551e-05 - accuracy: 1.0000
Test accuracy after structured pruning: 1.0


In [22]:
# 프루닝 레이어 제거 및 모델 저장
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
model_for_export.save('model_after_structured_pruning.h5')

In [23]:
# 모델 사이즈 측정
model_size_before = os.path.getsize('model_before_pruning.h5') / (1024 * 1024)
model_size_after = os.path.getsize('model_after_structured_pruning.h5') / (1024 * 1024)
print(f"Model Size before pruning: {model_size_before:.2f} MB")
print(f"Model Size after structured pruning: {model_size_after:.2f} MB")

Model Size before pruning: 1391.09 MB
Model Size after structured pruning: 463.71 MB


In [24]:
# 프루닝 후 비제로 가중치 계산
non_zero_after, total_after = calculate_non_zero_weights(model_for_export)
print(f"After Structured Pruning: Non-zero weights: {non_zero_after}/{total_after}")
print(f"Final non-zero weight ratio: {non_zero_after / total_after:.4f}")

After Structured Pruning: Non-zero weights: 12830488/121549684
Final non-zero weight ratio: 0.1056
